# ⚡⚡⚡ DisTilling Magic: NISQ's Revenge ⚡⚡⚡
### _Cover the Bloch Sphere before your T1 runs out! Look out for those coveted T gates!_

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from game import Game
game = Game()
canvas = game.play()
canvas